In [ ]:
%%HTML
<style>
.container { width: 100% }
</style>

# A Simple Symbolic Calculator

## Specification of the Scanner

In [ ]:
import ply.lex as lex

In [ ]:
tokens = [ 'NUMBER', 'IDENTIFIER' ]

In [ ]:
def t_NUMBER(t):
    r'0|[1-9][0-9]*(\.[0-9]+)?(e[+-]?([1-9][0-9]*))?'
    t.value = float(t.value)
    return t

In [ ]:
def t_IDENTIFIER(t):
    r'[a-zA-Z][a-zA-Z0-9_]*'
    return t

In [ ]:
literals = ['=', '+', '-', '*', '/', '(', ')', ';']

In [ ]:
t_ignore  = ' \t'

In [ ]:
def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

In [ ]:
def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

In [ ]:
__file__ = 'main'

In [ ]:
lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

In [ ]:
precedence = [ ('left', '+', '-'),
               ('left', '*', '/'),
               ('right', 'UMINUS')
             ]

In [ ]:
Names2Values = {}

In [ ]:
start = 'statement'

In [ ]:
def p_statement_assign(p):
    'statement : IDENTIFIER "=" expression'
    Names2Values[p[1]] = p[3]

In [ ]:
def p_statement_expr(p):
    'statement : expression'
    print(p[1])

In [ ]:
def p_expression_binary(p):
    '''expression : expression '+' expression
                  | expression '-' expression
                  | expression '*' expression
                  | expression '/' expression
    '''
    if p[2] == '+':
        p[0] = p[1] + p[3]
    elif p[2] == '-':
        p[0] = p[1] - p[3]
    elif p[2] == '*':
        p[0] = p[1] * p[3]
    elif p[2] == '/':
        p[0] = p[1] / p[3]
    elif len(p) == 2:
        p[0] = -p[2]

In [ ]:
def p_expression_uminus(p):
    "expression : '-' expression %prec UMINUS"
    p[0] = -p[2]

In [ ]:
def p_expression_group(p):
    "expression : '(' expression ')'"
    p[0] = p[2]

In [ ]:
def p_expression_number(p):
    "expression : NUMBER"
    p[0] = p[1]

In [ ]:
def p_expression_name(p):
    "expression : IDENTIFIER"
    try:
        p[0] = Names2Values[p[1]]
    except LookupError:
        print(f'Undefined name: {p[1]}')
        p[0] = 0

In [ ]:
def p_error(p):
    if p:
        print(f'Syntax error at {p.value}.')
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an obscure bug where the parser generator tries  to read an empty parse table.

In [ ]:
parser = yacc.yacc(write_tables=False)

In [ ]:
def main():
    while True:
        try:
            s = input('calc > ')
        except EOFError:
            break
        if s == 'quit':
            break
        if not s:
            continue
        yacc.parse(s)

In [ ]:
main()